In [5]:
print("Starting")
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

Starting


In [6]:
df = pd.read_xml("defects.xml")

In [12]:
df

,name,Background,Crack,Spallation,Efflorescence,ExposedBars,CorrosionStain,sum,text_base
0,image_0000005_crop_0000001.png,0,0,0,1,0,1,2,"Efflorescence, CorrosionStain"
1,image_0000005_crop_0000002.png,0,0,0,1,0,1,2,"Efflorescence, CorrosionStain"
2,image_0000021_crop_0000001.png,0,0,0,1,0,0,1,Efflorescence
3,image_0000028_crop_0000001.png,0,0,0,1,0,1,2,"Efflorescence, CorrosionStain"
4,image_0000028_crop_0000002.png,0,0,0,1,0,0,1,Efflorescence
...,...,...,...,...,...,...,...,...,...
5239,image_0001599_crop_0000004.png,0,0,0,0,0,1,1,CorrosionStain
5240,image_0001599_crop_0000005.png,0,0,1,0,1,1,3,"Spallation, ExposedBars, CorrosionStain"
5241,image_0001599_crop_0000006.png,0,0,1,0,1,1,3,"Spallation, ExposedBars, CorrosionStain"
5242,image_0001599_crop_0000007.png,0,0,0,0,0,1,1,CorrosionStain


In [8]:
df['sum'] = df['Crack'] + df['Spallation'] + df['Efflorescence'] + df['ExposedBars'] + df['CorrosionStain']
df.drop(['Total'], axis=1, inplace=True)
df.drop(df.index[-1],inplace=True)

In [10]:
text_list = []
for ind in df.index:
    temp_list = ""
    if(df['Crack'][ind]==1):
        temp_list += "Crack"
    if(df['Spallation'][ind]==1):
        temp_list += ", Spallation"
    if(df['Efflorescence'][ind]==1):
        temp_list += ", Efflorescence"
    if(df['ExposedBars'][ind]==1):
        temp_list += ", ExposedBars"
    if(df['CorrosionStain'][ind]==1):
        temp_list += ", CorrosionStain"
    text_list.append(temp_list)

In [11]:
df['text_base']  = ""
for ind in df.index:
    df['text_base'][ind] = text_list[ind]
for ind in df.index:
    if(len(df['text_base'][ind])==0):
        df['text_base'][ind] = "None"
    if(df['text_base'][ind][0]==','):
        df['text_base'][ind] = df['text_base'][ind][2:]

In [13]:
from transformers import CLIPProcessor, CLIPModel

model_id = "openai/clip-vit-base-patch32"

processor = CLIPProcessor.from_pretrained(model_id)
model = CLIPModel.from_pretrained(model_id)

c:\Users\Pupul\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
device

'cuda'

In [15]:
clip_labels = [f"a photo of a {label}" for label in df['text_base']]
clip_labels

['a photo of a Efflorescence, CorrosionStain',
 'a photo of a Efflorescence, CorrosionStain',
 'a photo of a Efflorescence',
 'a photo of a Efflorescence, CorrosionStain',
 'a photo of a Efflorescence',
 'a photo of a Efflorescence, CorrosionStain',
 'a photo of a Crack',
 'a photo of a Efflorescence',
 'a photo of a Crack',
 'a photo of a Efflorescence',
 'a photo of a Crack',
 'a photo of a Efflorescence, CorrosionStain',
 'a photo of a Efflorescence',
 'a photo of a Efflorescence, CorrosionStain',
 'a photo of a Crack, Efflorescence, CorrosionStain',
 'a photo of a Efflorescence',
 'a photo of a Efflorescence, CorrosionStain',
 'a photo of a CorrosionStain',
 'a photo of a Efflorescence',
 'a photo of a Crack',
 'a photo of a Efflorescence, CorrosionStain',
 'a photo of a Efflorescence, CorrosionStain',
 'a photo of a Efflorescence, CorrosionStain',
 'a photo of a Efflorescence, CorrosionStain',
 'a photo of a Efflorescence, CorrosionStain',
 'a photo of a Efflorescence, CorrosionSt

In [16]:
import os
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from collections import OrderedDict
import torch

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [17]:
path = "D:/SUDHANSU/Coding/Projects/CLIP/Concrete/Train"

In [18]:
import clip

In [19]:
AAA , preprocess = clip.load("ViT-B/32")

In [21]:
original_images = []
images = []
texts = []
plt.figure(figsize=(16, 5))
for filename in df['name']:
#for filename in [filename for filename in os.listdir(path) if filename.endswith(".png") or filename.endswith(".jpg")]:
    image = Image.open(os.path.join(path, filename)).convert("RGB")
    original_images.append(image)
    images.append(preprocess(image))

<Figure size 1600x500 with 0 Axes>

In [22]:
label_tokens = processor(
    text=clip_labels,
    padding=True,
    images=None,
    return_tensors='pt'
).to(device)

label_tokens['input_ids'][0][:10]

tensor([49406,   320,  1125,   539,   320, 20548,   549,  3956,   954,   267],
       device='cuda:0')

In [28]:
label_emb = model.get_text_features(**label_tokens)
# detach from pytorch gradient computation
label_emb = label_emb.detach().cpu().numpy()
label_emb.shape

OutOfMemoryError: CUDA out of memory. Tried to allocate 246.00 MiB (GPU 0; 4.00 GiB total capacity; 6.20 GiB already allocated; 0 bytes free; 6.38 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
label_emb.min(), label_emb.max()

In [ ]:
import numpy as np

# normalization
label_emb = label_emb / np.linalg.norm(label_emb, axis=0)
label_emb.min(), label_emb.max()

In [24]:
image = processor(
    text=None,
    images=original_images[0],
    return_tensors='pt'
)['pixel_values'].to(device)
image.shape

torch.Size([1, 3, 224, 224])

In [ ]:
img_emb = model.get_image_features(image)
img_emb.shape

In [ ]:
img_emb = img_emb.detach().cpu().numpy()

In [ ]:
scores = np.dot(img_emb, label_emb.T)
scores.shape

In [ ]:
# get index of highest score
pred = np.argmax(scores)
pred

In [ ]:
from tqdm.auto import tqdm

preds = []
batch_size = 32

for i in tqdm(range(0, len(original_images), batch_size)):
    i_end = min(i + batch_size, len(original_images))
    images = processor(
        text=None,
        images=original_images[i:i_end],
        return_tensors='pt'
    )['pixel_values'].to(device)
    img_emb = model.get_image_features(images)
    img_emb = img_emb.detach().cpu().numpy()
    scores = np.dot(img_emb, label_emb.T)
    preds.extend(np.argmax(scores, axis=1))

In [ ]:
true_preds = []
for i, label in enumerate(imagenette['label']):
    if label == preds[i]:
        true_preds.append(1)
    else:
        true_preds.append(0)

acc = sum(true_preds) / len(true_preds) * 100
print("Accuracy = ",acc)
print("CODEBRIM")